In [1]:
import numpy as np
import numpy.linalg as lia
import pandas as pd
import matplotlib as plt

In [2]:
from sklearn import datasets
digits = datasets.load_digits()

In [3]:
print(len(digits.data))
print(digits.target.size)

1797
1797


In [4]:
from sklearn.datasets import fetch_openml
wine = fetch_openml(name='wine', version=1)

In [5]:
print(len(wine.data))
print(wine.target.size)

178
178


In [6]:
#normalising wine data
wine_data_norm = []
for col in wine.data.T:
    col_norm = col/np.amax(col)
    wine_data_norm.append(col_norm)
    
wine.data = np.asarray(wine_data_norm).T

In [7]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []
    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index+(foldIndex*wineValidationSetSize)])
    
    xTrainingSet = []
    yTrainingSet = []
    
    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)


In [8]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [9]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [10]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=0.5, momentum=0.9, max_iters=20, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.epsilon = epsilon
            
    def run(self, gradient_fn, x, y, w):
        # TODO: use epsilon
        
        grad = np.inf
        t = 1
        
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            # TODO: implement momentum here
            
            print("gradient descent step:", t)
            
            gradients = gradient_fn(x, y, w, self.batch_size)
            
            for c in range(len(y[0])):
                w[c] = w[c] - self.learning_rate * gradients[c]
#                 print("w for class: ", w[c])
            
            t += 1
        
        return w

In [11]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        # TODO: add bias
        
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])
                
#                 print("x:", a.astype(int))
#                 print("y:", b)

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    
                    # TODO: may change, see slide 27 of logistic slideshow
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    gradients[c] += cost_c
                    
#                     print("class:", c)
#                     print("softmax numerator:", num)
#                     print("softmax denominator:", den)
#                     print("y hat for class:", yh_c)
#                     print("y actual for class:", y_c)
#                     print("x gradient:", cost_c)
#                     print("new gradient for class:", gradients[c])

            return gradients
        
        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
        return self
    
    def predict(self, x):
        a = np.asarray(x)
        b = np.asarray(self.w)
        
#         if self.add_bias:
#             x = np.column_stack([x,np.ones(N)])

        yh=[]
        for i,x_c in enumerate(a):
            yh_x=[]
            for c in range(len(b)):
                w_x =  b[c] @ x_c
                num = np.exp(w_x)

                den = 0
                for i in range(len(b)):
                    w_x =  b[i] @ x_c
                    den += np.exp(w_x)

                yh_c = num/den
                yh_x.append(yh_c)
            yh.append(yh_x)
        return yh

In [13]:



def cost(yh, y):
    return y * np.log1p(np.exp(-yh)) + (1-yh) * np.log1p(np.exp(yh))

for fold_index, fold in enumerate(xWineTrainingSets):
    gradientDescentModel = GradientDescent(10)
    logisticRegressionModel = LogisticRegression(False)
    
    logisticRegressionModel.fit(fold, yWineTrainingSets[fold_index], gradientDescentModel)
    yh = logisticRegressionModel.predict(xWineValidationSets[fold_index])
    
    for sample_index, yh_x in enumerate(yh):
        print(yh_x, yWineValidationSets[fold_index][sample_index])
        pass


# for i, yh_x in enumerate(yh):
    
#     print()
#     y_index = -1
#     for j, c in enumerate(yWineTrainingSets[0][i]):
#         if int(c) == 1:
#             y_index = j
            
#     print(y_index)
        
#     #cst = cost(yh_x)
#     print(yh_x, yWineTrainingSets[0][i])




#xWineTrainingSets = []
#yWineTrainingSets = []
#xWineValidationSets = []
#yWineValidationSets = []

gradient descent step: 1
gradient descent step: 2
gradient descent step: 3
gradient descent step: 4
gradient descent step: 5
gradient descent step: 6
gradient descent step: 7
gradient descent step: 8
gradient descent step: 9
gradient descent step: 10
gradient descent step: 11
gradient descent step: 12
gradient descent step: 13
gradient descent step: 14
gradient descent step: 15
gradient descent step: 16
gradient descent step: 17
gradient descent step: 18
gradient descent step: 19
[0.9949122315732559, 0.005087766932441624, 1.4943024604269124e-09] [1.0, 0.0, 0.0]
[0.9937763402665426, 0.006223647483410678, 1.2250046729543357e-08] [1.0, 0.0, 0.0]
[0.996826508395333, 0.0031734862538432035, 5.3508237167773514e-09] [1.0, 0.0, 0.0]
[0.9999319057699267, 6.809375572756675e-05, 4.743456760935382e-10] [1.0, 0.0, 0.0]
[0.9359907105153912, 0.06400896388630334, 3.2559830558754403e-07] [1.0, 0.0, 0.0]
[0.9997085166449766, 0.0002914799957747018, 3.3592486474551668e-09] [1.0, 0.0, 0.0]
[0.99765959166977

gradient descent step: 11
gradient descent step: 12
gradient descent step: 13
gradient descent step: 14
gradient descent step: 15
gradient descent step: 16
gradient descent step: 17
gradient descent step: 18
gradient descent step: 19
[1.3557525721040914e-06, 0.054908657402630914, 0.945089986844797] [0.0, 0.0, 1.0]
[1.904385260204277e-05, 0.024422186802755173, 0.9755587693446427] [0.0, 0.0, 1.0]
[5.561470478277989e-07, 0.0602024432896972, 0.939797000563255] [0.0, 0.0, 1.0]
[1.757729824046178e-06, 0.020122630628781224, 0.9798756116413947] [0.0, 0.0, 1.0]
[4.1779484590259285e-05, 0.0029426992123830187, 0.9970155213030267] [0.0, 0.0, 1.0]
[5.933222656255749e-06, 0.018092905932752975, 0.9819011608445907] [0.0, 0.0, 1.0]
[1.4420245444616145e-06, 0.007454252312763295, 0.9925443056626923] [0.0, 0.0, 1.0]
[2.5651416055919255e-06, 0.003200644704558639, 0.9967967901538358] [0.0, 0.0, 1.0]
[6.469151485945369e-06, 0.005661789653438617, 0.9943317411950754] [0.0, 0.0, 1.0]
[8.105494944403842e-06, 0.0